Network for arteficial data

In [21]:
import pandas as pd
import numpy as np
import torch
from scipy.integrate import odeint

In [22]:
data = pd.read_csv('/workspaces/bio-pinn/Arteficial Data/Data_noisy.csv')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
y0 = np.array([1., 0., 0., 0.])
print(data)


     time     ConcA     ConcB     ConcC     ConcD
0     0.0  0.999271  0.007767  0.006653 -0.021096
1     0.1  0.923688  0.103277  0.089222  0.022587
2     0.2  0.819228  0.180845  0.140493  0.030172
3     0.3  0.768460  0.256496  0.165251  0.088740
4     0.4  0.679087  0.313991  0.191811  0.111818
..    ...       ...       ...       ...       ...
96    9.6  0.131623  0.846857  0.333304  0.510606
97    9.7  0.142404  0.864158  0.327191  0.517452
98    9.8  0.138910  0.865797  0.345474  0.490065
99    9.9  0.122091  0.872590  0.334365  0.517707
100  10.0  0.133917  0.862295  0.344170  0.516223

[101 rows x 5 columns]


In [23]:
class Net(torch.nn.Module):
    
    def __init__(self, k):
        super().__init__()
        self.actf = torch.tanh
        self.f1 = torch.nn.Linear(1, 100)
        self.f2 = torch.nn.Linear(100, 100)
        self.f3 = torch.nn.Linear(100, 100)
        self.f4 = torch.nn.Linear(100, 4)
        
        self.k1 = torch.nn.Parameter(torch.tensor(k[0], device=device))
        self.k2 = torch.nn.Parameter(torch.tensor(k[1], device=device))
        self.k3 = torch.nn.Parameter(torch.tensor(k[2], device=device))
        self.k4 = torch.nn.Parameter(torch.tensor(k[3], device=device))

    def forward(self, x):
        x = self.actf(self.f1(x))
        x = self.actf(self.f2(x))
        x = self.actf(self.f3(x))
        x = self.f4(x)
        return x.squeeze()

from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self, in_tensor, out_tensor):
        self.inp = in_tensor
        self.out = out_tensor

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.out[idx]

In [24]:
from torch.autograd import grad

def phys_loss(t, C, model):
    dCdt = torch.autograd.grad(
        C, t,
        grad_outputs=torch.ones_like(C),
        create_graph=True
    )[0]

    A, B, Cc, D = C[:,0], C[:,1], C[:,2], C[:,3]

    fA = -model.k1 * A
    fB = model.k1 * A - model.k2 * B
    fC = model.k2 * B - model.k3 * Cc
    fD = model.k3 * Cc - model.k4 * D

    rhs = torch.stack([fA, fB, fC, fD], dim=1)
    return torch.mean((dCdt - rhs)**2)

In [25]:
k = [7., 10., .1, 3.]
from torch.optim import Adam
model_pinn = Net(k).to(device)

epochs = 1000
optimizer_pinn = Adam(model_pinn.parameters(), lr=0.1)
loss_fcn = torch.nn.MSELoss()
df=data[['ConcA','ConcB','ConcC','ConcD']]

train_in = torch.tensor(data['time'].values, dtype=torch.float32, requires_grad=True).view(-1, 1).to(device)
train_out = torch.tensor(df.values, dtype=torch.float32).to(device)
time=torch.tensor(data['time'].values, dtype=torch.float32)


for epoch in range(epochs):
    model_pinn.train()  # Ensure model is in training mode

    optimizer_pinn.zero_grad()
    pred = model_pinn(train_in)
    base_loss = loss_fcn(pred, train_out)
    phys_loss_value = phys_loss(train_in, pred, model_pinn)
    total_loss = base_loss + phys_loss_value
    total_loss.backward()
    optimizer_pinn.step()

    # Print losses for every epoch
    print(f'Epoch {epoch+1}/{epochs} | Loss: {total_loss.item():.4f} = {base_loss.item():.4f} (Base) + {phys_loss_value.item():.4f} (Physics)')






Epoch 1/1000 | Loss: 1.7249 = 0.2866 (Base) + 1.4383 (Physics)
Epoch 2/1000 | Loss: 76.8987 = 1.2856 (Base) + 75.6130 (Physics)
Epoch 3/1000 | Loss: 11920.1221 = 117.4118 (Base) + 11802.7100 (Physics)
Epoch 4/1000 | Loss: 1534.1049 = 21.2118 (Base) + 1512.8931 (Physics)
Epoch 5/1000 | Loss: 949.0542 = 7.2957 (Base) + 941.7585 (Physics)
Epoch 6/1000 | Loss: 4187.7139 = 37.4261 (Base) + 4150.2876 (Physics)
Epoch 7/1000 | Loss: 4114.3408 = 53.6584 (Base) + 4060.6826 (Physics)
Epoch 8/1000 | Loss: 1890.5814 = 50.6729 (Base) + 1839.9084 (Physics)
Epoch 9/1000 | Loss: 264.0648 = 39.7847 (Base) + 224.2801 (Physics)
Epoch 10/1000 | Loss: 531.7946 = 30.5559 (Base) + 501.2387 (Physics)
Epoch 11/1000 | Loss: 1702.7096 = 23.7424 (Base) + 1678.9672 (Physics)
Epoch 12/1000 | Loss: 2154.0461 = 15.9320 (Base) + 2138.1140 (Physics)
Epoch 13/1000 | Loss: 1488.9092 = 8.7755 (Base) + 1480.1337 (Physics)
Epoch 14/1000 | Loss: 495.4576 = 6.9648 (Base) + 488.4928 (Physics)
Epoch 15/1000 | Loss: 50.7284 = 12.